# Proyecto Final de Deep Learning

### Presentación

* **Título:** Detección de Vehículos desde Vistas Aéreas: Comparativa de los Modelos YOLOv12 y RT-DETR
* **Autor:** Jared Orihuela Contreras
* **Afiliación:** Phaw AI 2025 - Curso de Deep Learning

---

### Introducción

Este proyecto se centra en la **detección de vehículos desde una perspectiva aérea**, una tarea fundamental en aplicaciones como la monitorización del tráfico y la planificación urbana. El objetivo principal es evaluar y comparar el rendimiento de dos arquitecturas de última generación: **YOLOv12** y **RT-DETR**. La evaluación se llevará a cabo en un **conjunto de datos personalizado**, el cual se compone de 626 imágenes con anotaciones específicas para la clase 'Vehículo'.

El estudio se enfocará en comparar los modelos en términos de métricas clave como la **precisión media promedio (mAP)**, **precisión y _recall_**, y la **velocidad de inferencia**. Se evaluarán tres versiones de modelos: YOLOv12 entrenado desde cero, YOLOv12 con _fine-tuning_ y RT-DETR con _fine-tuning_. La meta es determinar qué modelo ofrece el mejor equilibrio entre precisión y eficiencia para la detección de vehículos en este escenario. El proyecto culminará con una presentación de resultados mediante tablas y gráficos comparativos, identificando el modelo con el mejor desempeño.

In [1]:
!pip install ultralytics torch torchvision numpy matplotlib opencv-python -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.1 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.3 MB/s eta 0:00:00:00:0100:01


In [2]:
import ultralytics
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import cv2
import json
import os
import tqdm

from ultralytics import YOLO, RTDETR
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from matplotlib.patches import Rectangle
from PIL import Image
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from torchvision.datasets import CocoDetection
from tqdm import tqdm

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Preparación de Data
---
Cabe recalcar que para este proyecto evaluaremos las capacidades del modelos sobre los datos de validación, debido a que en el Dataset no se tiene los labels de la data de testeo.

In [3]:
instance_y = "/kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/train/images"
instance_x = "/kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/train/images"

## Evaluación

### Métricas clave

Para evaluar el rendimiento de un modelo de detección de objetos, se utilizan métricas específicas que miden tanto la precisión de la caja como la clasificación.

* **`mAP@0.5`:** Es la métrica más común. El **mAP** (_mean Average Precision_) te da un puntaje promedio de la precisión del modelo en todas las clases. El `@0.5` significa que una detección se considera correcta si el área de solapamiento entre la caja predicha y la caja real es de al menos
el 50%.

* **`mAP@[.5:.95]`:** Esta métrica es más estricta. Evalúa el mAP promediando los resultados en diferentes umbrales de solapamiento (IoU), desde 0.5 hasta 0.95, con pasos de 0.05. Es una excelente manera de medir la robustez del modelo.


* **`Precisión`:** La precisión responde a la pregunta: **"De todas las predicciones positivas que hizo mi modelo, ¿cuántas fueron realmente correctas?"** Se enfoca en la **calidad** de las detecciones. Un modelo con alta precisión tiene pocos **falsos positivos** (predicciones incorrectas). La precisión es crucial en aplicaciones donde las falsas alarmas son costosas o peligrosas, como la detección de enfermedades o la vigilancia.

  **`Fórmula`:** $$Precision = \frac{True\ Positives}{True\ Positives + False\ Positives}$$

* **`Recall`:** El _recall_ responde a la pregunta: **"De todos los objetos que realmente existían, ¿cuántos logró detectar mi modelo?"**, Se enfoca en la **cantidad** de objetos que el modelo es capaz de encontrar. Un modelo con alto _recall_ tiene pocos **falsos negativos** (objetos reales que no fueron detectados). El _recall_ es vital en aplicaciones donde no detectar un objeto es un riesgo mayor, como la detección de peatones en la conducción autónoma o de tumores en imágenes médicas.

  **`Fórmula`:** $$Recall = \frac{True\ Positives}{True\ Positives + False\ Negatives}$$

* **`Velocidad de inferencia`:** Mide el número de imágenes que el modelo puede procesar por segundo. Es una métrica crítica, especialmente para proyectos que requieren ser rápidos.


---
**IMPORTANTE!**
Para los siguientes modelos se necesitará un "yaml" file, con la información de los Dataset, es requerido por los modelos provenientes de la libreria Ultralytics

In [4]:
coco_file = "/kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/data.yaml"

### YOLO V12 (2025)

In [11]:
yolo_model = "yolo12s"
yolo12_model = YOLO(yolo_model)

In [16]:
yolo12_metrics = yolo12_model.val(data = coco_file)

Ultralytics 8.3.189 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv12s summary (fused): 159 layers, 9,261,840 parameters, 0 gradients, 21.4 GFLOPs
val: Fast image access ✅ (ping: 0.5±0.2 ms, read: 146.7±44.0 MB/s, size: 53.2 KB)
val: Scanning /kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/valid/labels... 90 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 90/90 994.2it/s 0.1ss
WARNING ⚠️ val: Cache directory /kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/valid is not writeable, cache not saved.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 1.9it/s 3.1ss


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         90        937     0.0268       0.08     0.0145    0.00816
                person         90        937     0.0268       0.08     0.0145    0.00816
Speed: 3.8ms preprocess, 12.5ms inference, 0.0ms loss, 9.7ms postprocess per image
Results saved to runs/detect/val


In [17]:
print(yolo12_metrics.results_dict)

{'metrics/precision(B)': 0.02682403433476395, 'metrics/recall(B)': 0.08004268943436499, 'metrics/mAP50(B)': 0.014511753874476126, 'metrics/mAP50-95(B)': 0.008158172055664884, 'fitness': 0.008793530237546008}


### YOLO V12 (Finetunned)

In [12]:
yolo12_finetuned_model = YOLO(yolo_model)

In [13]:
results_yolov12_train = yolo12_finetuned_model.train(data = coco_file, epochs = 10, imgsz = 640, batch = 4)

Ultralytics 8.3.189 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, ove

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         90        937       0.89      0.931      0.967      0.706
Speed: 0.2ms preprocess, 9.1ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train3


In [14]:
yolo12_finetuned_metrics = yolo12_finetuned_model.val(data = coco_file)

Ultralytics 8.3.189 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv12s summary (fused): 159 layers, 9,231,267 parameters, 0 gradients, 21.2 GFLOPs
val: Fast image access ✅ (ping: 1.2±0.7 ms, read: 100.1±42.6 MB/s, size: 61.7 KB)
val: Scanning /kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/valid/labels... 90 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 90/90 950.4it/s 0.1s
WARNING ⚠️ val: Cache directory /kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/valid is not writeable, cache not saved.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 23/23 12.3it/s 1.9s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         90        937       0.89      0.931      0.967      0.706
Speed: 0.9ms preprocess, 15.2ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train32


In [15]:
yolo12_finetuned_metrics.results_dict

{'metrics/precision(B)': 0.8898066505206459,
 'metrics/recall(B)': 0.9307316265221801,
 'metrics/mAP50(B)': 0.9672328277319089,
 'metrics/mAP50-95(B)': 0.7057710848472812,
 'fitness': 0.731917259135744}

### RT-DETR (2023)

In [5]:
rtdetr_finetuned_model = RTDETR("rtdetr-l.pt")

In [6]:
results_rtdetr_train = rtdetr_finetuned_model.train(data = coco_file, epochs=10, imgsz=640, batch = 4)

Ultralytics 8.3.189 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=rtdetr-l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, ove

/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       1/10      3.56G     0.5137     0.8122     0.1338         21        640: 100% ━━━━━━━━━━━━ 134/134 2.2it/s 1:02s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 4.0it/s 3.0s
                   all         90        937       0.83      0.851      0.894      0.632

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
       2/10      3.84G     0.3512     0.4228    0.09535         49        640:   0% ──────────── 0/134  0.4s

/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       2/10      3.84G     0.3437     0.4431    0.07475         36        640: 100% ━━━━━━━━━━━━ 134/134 2.3it/s 59.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 5.5it/s 2.2s
                   all         90        937      0.857       0.92      0.939      0.694

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
       3/10      3.93G     0.3503     0.4772    0.08357         46        640:   0% ──────────── 0/134  0.4s

...

       9/10      5.06G     0.2833     0.3822    0.05624         47        640: 100% ━━━━━━━━━━━━ 134/134 2.3it/s 58.5s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 5.5it/s 2.2s
                   all         90        937      0.878      0.926      0.948      0.718

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      10/10       5.2G     0.2679     0.3567    0.06253         41        640:   0% ──────────── 0/134  0.4s

/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      10/10      5.25G     0.2833     0.3786    0.05978          9        640: 100% ━━━━━━━━━━━━ 134/134 2.3it/s 58.6s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 5.5it/s 2.2s
                   all         90        937      0.881       0.92      0.939      0.714

10 epochs completed in 0.174 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 66.1MB
Optimizer stripped from runs/detect/train/weights/best.pt, 66.1MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.189 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
rt-detr-l summary: 302 layers, 31,985,795 parameters, 0 gradients, 103.4 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 3.9it/s 3.1s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         90        937      0.877      0.926      0.948      0.718
Speed: 0.2ms preprocess, 30.5ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/train


In [7]:
!zip -r train.zip /kaggle/working/runs/detect/train

  adding: kaggle/working/runs/detect/train/ (stored 0%)
  adding: kaggle/working/runs/detect/train/results.png (deflated 7%)
  adding: kaggle/working/runs/detect/train/BoxP_curve.png (deflated 16%)
  adding: kaggle/working/runs/detect/train/BoxF1_curve.png (deflated 15%)
  adding: kaggle/working/runs/detect/train/args.yaml (deflated 52%)
  adding: kaggle/working/runs/detect/train/train_batch2.jpg (deflated 4%)
  adding: kaggle/working/runs/detect/train/val_batch1_pred.jpg (deflated 2%)
  adding: kaggle/working/runs/detect/train/train_batch1.jpg (deflated 8%)
  adding: kaggle/working/runs/detect/train/results.csv (deflated 60%)
  adding: kaggle/working/runs/detect/train/weights/ (stored 0%)
  adding: kaggle/working/runs/detect/train/weights/last.pt (deflated 8%)
  adding: kaggle/working/runs/detect/train/weights/best.pt (deflated 8%)
  adding: kaggle/working/runs/detect/train/val_batch0_pred.jpg (deflated 1%)
  adding: kaggle/working/runs/detect/train/labels.jpg (deflated 34%)
  adding:

In [9]:
rtdetr_metrics = rtdetr_finetuned_model.val(data = coco_file)

Ultralytics 8.3.189 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
rt-detr-l summary: 302 layers, 31,985,795 parameters, 0 gradients, 103.4 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.0 ms, read: 136.6±34.2 MB/s, size: 61.7 KB)
val: Scanning /kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/valid/labels... 90 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 90/90 994.5it/s 0.1ss
WARNING ⚠️ val: Cache directory /kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/valid is not writeable, cache not saved.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 23/23 7.6it/s 3.0s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         90        937      0.877      0.926      0.948      0.719
Speed: 1.4ms preprocess, 27.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train2


In [10]:
rtdetr_metrics.results_dict

{'metrics/precision(B)': 0.8771930562193498,
 'metrics/recall(B)': 0.9263607257203842,
 'metrics/mAP50(B)': 0.9482223044288002,
 'metrics/mAP50-95(B)': 0.7194915127172211,
 'fitness': 0.742364591888379}